Hi there, I'm Jasmine from Santiago Canyon College, CA!

Let me informally walk you through the ideas of my paper: **Randomly Picking The Correct Number Somehow? An Introduction Into The Probabilistic Method**.

More specifically, I will try to implement different ideas and algorithms through out this notebook!

I hope you enjoy!


## The Naive Liar Game

Let's talk about the naive Liar Game when k = 0.

### Winning Condition

Given a range of $n$ numbers and $q$ queries, knowing that $k$ = 0, we want to compute if $n \le 2^q$ so that Lucy can win the game

In [50]:

# nv: naive
# LG: Liar Game
# Lcond: Lucy's win condition
def nv_LG_Lcond(n: int, q: int) -> bool:
    # print(f"The value of n is: {n}")
    # print(f"The value of 2^q is: {pow(2,q)}")
    return n <= pow(2,q)

# Check if Lucy have a winning strategy in the Liar Game of n=4, q=2, k=0
print(nv_LG_Lcond(4,2))

# Check if Lucy have a winning strategy in the Liar Game of n=10, q=3, k=0
print(nv_LG_Lcond(10,3))

True
False


### Winning Strategy

In the same theorem that provides us with a way to deduce the win condition for the naive Liar Game, its proof also gives us the hint to achieve a win for Lucy.

Let's code up the algorithm for this!

Let *A* be the first half of the remaining set.

Let *B* be the second half of the remaining set.

Pseudo-algorithm
1. Check the winning condition for Lucy
2. If Lucy can apply the strategy, continue
1. Query the set *A*
2. If Jasmine answers Yes, take set *A* to be the remaining set
3. If Jasmine answers No, take set *B* to be the remaining set
4. Repeat from step 2 until the cardinality of the remaining set is 1


In [56]:
class LG:
    # the boolean value that keeps the game going
    keepGoing = True

    # reset after a game is played
    def reset():
        keepGoing = True

    # nv: naive
    # LG: Liar Game
    # Lcond: Lucy's win condition
    def nv_LG_Lcond(n: int, q: int) -> bool:
        # print(f"The value of n is: {n}")
        # print(f"The value of 2^q is: {pow(2,q)}")
        return n <= pow(2,q)

    # sem: stands for start, end, mid reassignment function
    def sem(start, end):
        return start, end, int((start+end)/2)

    # logic: the function that helps decide the logic after Lucy queries and Jasmine gives the answers
    def logic(start, end, mid, isTrue):
        if(isTrue == True):
            if(start==mid):
                LG.keepGoing = False
                return start, end, mid
            return LG.sem(start, mid)
        if(isTrue == False):
            return LG.sem(mid+1,end)

    # nv: naive
    # LG: Liar Game
    # Lstrat: Lucy's winning strategy
    def nv_LG_Lstrat(n: int, q: int, x: int=-1):

        if(LG.nv_LG_Lcond(n,q) == False):
            print("Lucy is unable to deduce a winning strategy")
            LG.reset()
            return
            
        start, end, mid = LG.sem(1,n)
        while LG.keepGoing:
            if(x < 0):
                isTrue = input(f"Is x in the set from {start} to {mid} (y/n)").lower().strip() == 'y'
            else:
                print(f"Is x in the set from {start} to {mid} (y/n)")
                isTrue = (x >= start and x <= mid)
                print(f"Jasmine answers {isTrue}")
            start, end, mid = LG.logic(start, end, mid, isTrue)
            n = n / 2
            LG.keepGoing = LG.keepGoing and n > 1
        print(f"Answer is {mid}")
        LG.reset()
        return



    
LG.nv_LG_Lstrat(4,2,3)

# Readers can try to play against the algorithm by omitting the third parameter, 
# or making the parameter negative

# LG.nv_LG_Lstrat(4,2,-1)
# LG.nv_LG_Lstrat(4,2)

Is x in the set from 1 to 2 (y/n)
Jasmine answers False
Is x in the set from 3 to 3 (y/n)
Jasmine answers True
Answer is 3


## The Chip-Liar Games

Let us try our hands at the $(x_0, x_1, ..., x_k )$-Chip-Liar Game.

After we develop every concept for the $(x_0, x_1, ..., x_k )$-Chip-Liar Game, we can look back at the nqk-Chip-Liar Game in a new light!

### $B(q,j) = 2^{-q} \sum_{i=0}^j {q \choose i}$

Let us implement this idea that is integral in both our winning condition and winning strategy of Jasmine in the Chip-Liar Game.

In [37]:
import math

def B(q: int, j: int) -> float:
    sum = 0.0
    for i in range(0,j+1):
        sum += math.comb(q,i)
    
    return float(1.0*sum*pow(2,-q))

# Write out some B(q,j) and use desmos to test out
print(B(4,2))


0.6875


### Winning Condition



In [13]:
import math
class CL:
    def __init__(self):
        pass

    def B(q: int, j: int) -> float:
        sum = 0.0
        for i in range(0,j+1):
            sum += math.comb(q,i)
        return float(1.0*sum*pow(2,-q))

    def get_input(q: int, inputList: None):
        if (q<0):
            # Prompt user to enter a list of integer of length k
            inputList = [int(item) for item in input("Enter a list of integers: ").split()]

            # Prompt user to enter a integer value q
            q = int(input("Enter a value for q: "))
        return q, inputList


    # mult: stands for (x0, x1,..., xk) game
    # CL : Chip-Liar
    # Jcond: Jasmine win condition
    def mult_CL_Jcond(q = -1, inputList = None):

        # If the user hasn't specified any input value
        if(q < 0):
            q, inputList = CL.get_input(q, inputList)
        """
        print("Current list: ")
        print(self.inputList)
        print(f"From the list's length, the value of k is {k}")
        print(f"q is {self.q}")
        """
        sum = 0
        for i, val in enumerate(inputList):
            sum = sum + val*CL.B(q,i)
        
        return sum

    
    def get_new_list(isTrue, queryList, inputList):
        # queryList stands for the set S
        # ls stands for the set S'
        ls = [inputList[i] - queryList[i] for i in range(len(inputList))]
        if(isTrue):
            queryList = queryList[1:] + [0]
            return [ls[i] + queryList[i] for i in range(len(inputList))]
        else:
            ls = ls[1:] + [0]
            return [queryList[i] + ls[i] for i in range(len(inputList))]




    def mult_CL_Jstrat(q = -1, inputList = None):
        q, inputList = CL.get_input(q, inputList)


        tempList = list(inputList)
        tempQ = q
        print("Current list: ")
        print(tempList)
        while(tempQ > 0):
            query = [int(item) for item in input(f"Enter a list of integers of length {len(tempList)} to query as Lucy: ").split()]
            print("Lucy queries: ")
            print(query)
            yesList = CL.get_new_list(True, query, tempList)
            yesWeight = CL.mult_CL_Jcond(tempQ,yesList)

            noList = CL.get_new_list(False, query, tempList)
            noWeight = CL.mult_CL_Jcond(tempQ, noList)

            if(yesWeight >= noWeight):
                print("Jasmine answers Yes!")
                tempList = yesList
            else:
                print("Jasmine answers No!")
                tempList = noList
            print("Updated list")
            print(tempList)
            tempQ = tempQ - 1
        pass

q = 2
ls = [1,2, 3]
print(CL.mult_CL_Jcond(2, ls) > 1)

print(CL.get_new_list(False,[1,1], [1,2]))

CL.mult_CL_Jstrat(4,[0,0,3])

True
[2, 1]
Current list: 
[0, 0, 3]
Lucy queries: 
[0, 0, 3]
Jasmine answers No!
Updated list
[0, 0, 3]
Lucy queries: 
[0, 0, 1]
Jasmine answers Yes!
Updated list
[0, 1, 2]
Lucy queries: 
[0, 0, 2]
Jasmine answers Yes!
Updated list
[0, 3, 0]
Lucy queries: 
[0, 1, 0]
Jasmine answers Yes!
Updated list
[1, 2, 0]
